In [ ]:
# Transformers installation
! pip install transformers[torch] datasets evaluate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [2]:
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

In [3]:
notebook_login()

# Training

In [4]:
DATA = "imdb"
MODEL = "bert-base-uncased"
METRICS = ["accuracy", "f1"]
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [5]:
def preprocess_function(sents):
    return tokenizer(sents["text"], truncation=True)

In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {M: metrics[M].compute(predictions=predictions, references=labels) for M in metrics}

In [7]:
imdb = load_dataset(DATA)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
data = imdb.map(preprocess_function, batched=True)
metrics = {M: evaluate.load(M) for M in METRICS}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL, num_labels=2, id2label=id2label, label2id=label2id
)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir="modlee_transformer",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/harshitakukreja/modlee_transformer into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.207900,0.237863,{'accuracy': 0.9226},{'f1': 0.9261929282526605}
2,0.138200,0.217038,{'accuracy': 0.94184},{'f1': 0.9419838799776555}


Trainer is attempting to log a value of "{'accuracy': 0.9226}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9261929282526605}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.94184}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9419838799776555}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=3126, training_loss=0.19395429204841952, metrics={'train_runtime': 6205.9355, 'train_samples_per_second': 8.057, 'train_steps_per_second': 0.504, 'total_flos': 1.303899457047552e+16, 'train_loss': 0.19395429204841952, 'epoch': 2.0})

In [9]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Aug13_18-00-03_62f57f7026df/events.out.tfevents.1691949613.62f57f7026df.181.0:   0%|         …

To https://huggingface.co/harshitakukreja/modlee_transformer
   15a8e01..467b4f4  main -> main

   15a8e01..467b4f4  main -> main

To https://huggingface.co/harshitakukreja/modlee_transformer
   467b4f4..2b1d620  main -> main

   467b4f4..2b1d620  main -> main



'https://huggingface.co/harshitakukreja/modlee_transformer/commit/467b4f499d68f3f7eba1ed9385a4d1387b5864e2'

# Inference

In [4]:
from transformers import pipeline

text = "Hello"
classifier = pipeline("sentiment-analysis", model="harshitakukreja/modlee_transformer")
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.6614580154418945}]